# Linear programming - A Blending Problem

Follows the Whiskas example from the [Pulp documentation](https://coin-or.github.io/pulp/CaseStudies/a_blending_problem.html)

<img src="https://coin-or.github.io/pulp/_images/whiskas_label.jpg" width="450"/>   

<img src="https://coin-or.github.io/pulp/_images/whiskas_blend.jpg" width="400"/>   


## Problem Description
Whiskas cat food, shown above, is manufactured by Uncle Ben’s. Uncle Ben’s want to produce their cat food products as cheaply as possible while ensuring they meet the stated nutritional analysis requirements shown on the cans.    
Thus they want to vary the quantities of each ingredient used (the main ingredients being chicken, beef, mutton, rice, wheat and gel) while still meeting their nutritional standards

Reading of the nutritional requirements on the can in the image gives in $g/100g$:
- minimum protein >= 8
- minimum fat >= 6
- maximum fibre <= 2
- maximum salt <= 0.4

The costs of the chicken, beef, and mutton are $0.013, $0.008 and $0.010 respectively, while the costs of the rice, wheat and gel are $0.002, $0.005 and $0.001 respectively. (All costs are per gram.) For this exercise we will ignore the vitamin and mineral ingredients. (Any costs for these are likely to be very small anyway.)

Each ingredient contributes to the total weight of protein, fat, fibre and salt in the final product. The contributions (in grams) per gram of ingredient are given in the table below.
| Stuff       | Protein | Fat   | Fibre | Salt |
|-------------|---------|-------|-------|------|
| Chicken     | 0.100   | 0.080 | 0.001 | 0.002|
| Beef        | 0.200   | 0.100 | 0.005 | 0.005|
| Mutton      | 0.150   | 0.110 | 0.003 | 0.007|
| Rice        | 0.000   | 0.010 | 0.100 | 0.002|
| Wheat bran  | 0.040   | 0.010 | 0.150 | 0.008|
| Gel         | 0.000   | 0.000 | 0.000 | 0.000|


## 1. Simplified Formulation
First we will consider a simplified problem to build a simple Python model.

##### Identify the Decision Variables  
Assume Whiskas want to make their cat food out of just two ingredients: Chicken and Beef. We will first define our decision variables:

- $x_1 =$ percentage of chicken meat in a can of cat food
- $x_2 =$ percentage of beef meat in a can of cat food

 
The limitations on these variables (greater than zero) must be noted but for the Python implementation, they are not entered or listed separately or with the other constraints.

##### Formulate the Objective Function  
The objective function becomes:

**Minimize**
- $z = 0.013*x_1 + 0.008*x_2$

##### Constraints
The constraints on the variables are that they must sum to 100 and that the nutritional requirements are met:

- $1.000*x_1 + 1.000*x_2 = 100.0$   # total grams
- $0.100*x_1 + 0.200*x_2 >= 8$   # total protein
- $0.080*x_1 + 0.100*x_2 >= 6$   # total fat
- $0.001*x_1 + 0.005*x_2 <= 2$   # total fibre
- $0.002*x_1 + 0.005*x_2 <= 0.4$   # total salt

## 2. Solving the simplified problem with PuLP

### 2.1 Import PuLP modeler functions

In [1]:
# imports
from pulp import LpMaximize, LpMinimize, LpProblem, LpStatus, lpSum, LpVariable

In [2]:
# Initialize linear problem
model = LpProblem("blending_problem", LpMinimize)
model

blending_problem:
MINIMIZE
None
VARIABLES

In [3]:
# Decision variables
x1 = LpVariable(name="x1", lowBound=0, cat='Continuous') # % chicken meat
x2 = LpVariable(name="x2", lowBound=0, cat='Continuous') # % beef meat
x1

x1

In [4]:
# Objective function
obj_func = 0.013*x1 + 0.008*x2
print(obj_func, "\n")

model += obj_func, "Total Cost of Ingredients per can"
print(model)

0.013*x1 + 0.008*x2 

blending_problem:
MINIMIZE
0.013*x1 + 0.008*x2 + 0.0
VARIABLES
x1 Continuous
x2 Continuous



In [5]:
# Constraints
model += (x1 + x2 == 100, "sum_to_100%")
model += (0.100*x1 + 0.200*x2 >= 8.0, "protein_requirement")
model += (0.080*x1 + 0.100*x2 >= 6.0, "fat_requirement")
model += (0.001*x1 + 0.005*x2 <= 2.0, "fiber_requirement")
model += (0.002*x1 + 0.005*x2 <= 0.4, "salt_requirement")
model

blending_problem:
MINIMIZE
0.013*x1 + 0.008*x2 + 0.0
SUBJECT TO
sum_to_100%: x1 + x2 = 100

protein_requirement: 0.1 x1 + 0.2 x2 >= 8

fat_requirement: 0.08 x1 + 0.1 x2 >= 6

fiber_requirement: 0.001 x1 + 0.005 x2 <= 2

salt_requirement: 0.002 x1 + 0.005 x2 <= 0.4

VARIABLES
x1 Continuous
x2 Continuous

Now that all the problem data is entered, the writeLP() function can be used to copy this information into a .lp file into the directory that your code-block is running from. Once your code runs successfully, you can open this .lp file with a text editor to see what the above steps were doing

In [6]:
# The problem data is written to an .lp file
model.writeLP("WhiskasModel.lp")

[x1, x2]

In [7]:
# Solve the optimization problem
status = model.solve()
status

print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for var in model.variables():
    print(f"{var.name}: {var.value()}")
for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 0.966666665
x1: 33.333333
x2: 66.666667
sum_to_100%: 1.4210854715202004e-14
protein_requirement: 8.6666667
fat_requirement: 3.3333333400000007
fiber_requirement: -1.6333333319999999
salt_requirement: 1.0000000272292198e-09


Interpretation of results:   
- ```status: 1, Optimal``` =>  Solver found an optimal solution
- ```objective: 0.9666``` => Objective function's optimal value is 0.967. (Here total price of 0.96 cents per can)
- ```x1: 33.333``` => Optimal value for decision variable x1 (chicken) = 33.33% 
- ```x2: 66.666``` => Optimal value for decision variable x2 (beef) = 66.667% 

Slack variables of the constraints:
- ```sum_to_100%: 1.421004e-14``` =>  **neligible** slack in the sum_to_100% constraint.   
 ($100 + 0 = 100.00g$)
 - ```protein_requirement: 8.666``` => 8.66g of **additional** protein is left over after the protein_requirement constraint is met.   
 ($8 + 8.6666667 = 16.6666667g$)
 - ```fat_requirement: 3.333``` => 3.33g of **additional** fat is left over after the fat_requirement constraint is met.   
 ($6 + 3.33 = 9.33g$)
 - ```fibre_requirement: -1.633``` => 1.63g of fibre **exceeding** the fibre_requirement constraint.   
 ($2 - (-1.633) = 3.633g$)
 - ```salt_requirement: 1.00e-09``` => **neligible** surplus in the salt_requirement constraint.  
 ($0.4 - 0 = 0.4g$)

Slack variables don't mean the resulting value for the constraints, but rather tells us the amount of slack (or surplus) in each constraint from the goal defined in the constraint. Think of it as distance from defined goal.


In [8]:
model.solver

Above we see that PuLP choose the CBC solver since we didn't specify one.   
Next we try the solver from the GLPK package.

In [9]:
from pulp import GLPK

# solving with GLPK - (got practically same results)
status = model.solve(solver=GLPK(msg=False)) # specify GLPK solver
status

print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for var in model.variables():
    print(f"{var.name}: {var.value()}")
for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 0.9666665000000001
x1: 33.3333
x2: 66.6667
sum_to_100%: 1.4210854715202004e-14
protein_requirement: 8.666670000000002
fat_requirement: 3.333334000000001
fiber_requirement: -1.6333332
salt_requirement: 1.0000000005838672e-07


## 2. Full Formulation

One can of cat food is 100g so the amounts is the same for grams or percents in this case.

#### Decision Variables
- $x_1 =$ percentage of chicken meat in a can of cat food
- $x_2 =$ percentage of beef meat in a can of cat food
- $x_3 =$ percentage of mutton meat in a can of cat food
- $x_4 =$ percentage of rice in a can of cat food
- $x_5 =$ percentage of wheat bran in a can of cat food
- $x_6 =$ percentage of gel in a can of cat food

Note that these percentages must be between 0 and 100.

#### Objective Function
The objective is to minimise the total cost of ingredients per can of cat food.   

**Minimize**  

$z = 0.013*x_1 + 0.008*x_2 + 0.010*x_3 + 0.002*x_4 + 0.005*x_5 + 0.001*x_6$

#### Constraints
- $x_1 + x_2 + x_3 + x_4 + x_5 + x_6 == 100$ # total grams
- $0.100*x_1 + 0.200*x_2 + 0.150*x_3 + 0.000*x_4 + 0.040*x_5 + 0.000*x_6 >= 8$ # min protein
- $0.080*x_1 + 0.100*x_2 + 0.110*x_3 + 0.010*x_4 + 0.010*x_5 + 0.000*x_6 >= 6$ # min fat
- $0.001*x_1 + 0.005*x_2 + 0.003*x_3 + 0.100*x_4 + 0.150*x_5 + 0.000*x_6 <= 2$ # max fibre
- $0.002*x_1 + 0.005*x_2 + 0.007*x_3 + 0.002*x_4 + 0.008*x_5 + 0.000*x_6 <= 0.4$ # max salt


In [10]:
modelFull = LpProblem("WhiskasFull_problem", LpMinimize)
modelFull

WhiskasFull_problem:
MINIMIZE
None
VARIABLES

In [11]:
# Decision variables - writing it fully out
x1 = LpVariable(name="x1_chicken", lowBound=0, cat='Continuous') # % chicken meat
x2 = LpVariable(name="x2_beef", lowBound=0, cat='Continuous')    # % beef meat
x3 = LpVariable(name="x3_mutton", lowBound=0, cat='Continuous')  # % mutton meat
x4 = LpVariable(name="x4_rice", lowBound=0, cat='Continuous')    # % rice
x5 = LpVariable(name="x5_wheat", lowBound=0, cat='Continuous')   # % wheat
x6 = LpVariable(name="x6_gel", lowBound=0, cat='Continuous')     # % gel
x1

x1_chicken

In [12]:
# Objective function
obj_func = 0.013*x1 + 0.008*x2 + 0.010*x3 + 0.002*x4 + 0.005*x5 + 0.001*x6
print(obj_func, "\n")

modelFull += obj_func # add objective function to the model
print(modelFull)


0.013*x1_chicken + 0.008*x2_beef + 0.01*x3_mutton + 0.002*x4_rice + 0.005*x5_wheat + 0.001*x6_gel 

WhiskasFull_problem:
MINIMIZE
0.013*x1_chicken + 0.008*x2_beef + 0.01*x3_mutton + 0.002*x4_rice + 0.005*x5_wheat + 0.001*x6_gel + 0.0
VARIABLES
x1_chicken Continuous
x2_beef Continuous
x3_mutton Continuous
x4_rice Continuous
x5_wheat Continuous
x6_gel Continuous



### Constraints reminder
- $x_1 + x_2 + x_3 + x_4 + x_5 + x_6 == 100$ # total grams
- $0.100*x_1 + 0.200*x_2 + 0.150*x_3 + 0.000*x_4 + 0.040*x_5 + 0.000*x_6 >= 8$ # min protein
- $0.080*x_1 + 0.100*x_2 + 0.110*x_3 + 0.010*x_4 + 0.010*x_5 + 0.000*x_6 >= 6$ # min fat
- $0.001*x_1 + 0.005*x_2 + 0.003*x_3 + 0.100*x_4 + 0.150*x_5 + 0.000*x_6 <= 2$ # max fibre
- $0.002*x_1 + 0.005*x_2 + 0.007*x_3 + 0.002*x_4 + 0.008*x_5 + 0.000*x_6 <= 0.4$ # max salt

In [13]:
# Constraints
modelFull += (x1 + x2 + x3 + x4 + x5 + x6 == 100, "sum_to_100%")
modelFull += (0.100*x1 + 0.200*x2 + 0.150*x3 + 0.000*x4 + 0.040*x5 + 0.000*x6 >= 8.0, "protein_req") # min protein
modelFull += (0.080*x1 + 0.100*x2 + 0.110*x3 + 0.010*x4 + 0.010*x5 + 0.000*x6 >= 6.0, "fat_req") # min fat
modelFull += (0.001*x1 + 0.005*x2 + 0.003*x3 + 0.100*x4 + 0.150*x5 + 0.000*x6 <= 2.0, "fibre_req") # max fibre
modelFull += (0.002*x1 + 0.005*x2 + 0.007*x3 + 0.002*x4 + 0.008*x5 + 0.000*x6 <= 0.4, "salt_req") # max salt
modelFull

WhiskasFull_problem:
MINIMIZE
0.013*x1_chicken + 0.008*x2_beef + 0.01*x3_mutton + 0.002*x4_rice + 0.005*x5_wheat + 0.001*x6_gel + 0.0
SUBJECT TO
sum_to_100%: x1_chicken + x2_beef + x3_mutton + x4_rice + x5_wheat + x6_gel
 = 100

protein_req: 0.1 x1_chicken + 0.2 x2_beef + 0.15 x3_mutton + 0.04 x5_wheat
 >= 8

fat_req: 0.08 x1_chicken + 0.1 x2_beef + 0.11 x3_mutton + 0.01 x4_rice
 + 0.01 x5_wheat >= 6

fibre_req: 0.001 x1_chicken + 0.005 x2_beef + 0.003 x3_mutton + 0.1 x4_rice
 + 0.15 x5_wheat <= 2

salt_req: 0.002 x1_chicken + 0.005 x2_beef + 0.007 x3_mutton + 0.002 x4_rice
 + 0.008 x5_wheat <= 0.4

VARIABLES
x1_chicken Continuous
x2_beef Continuous
x3_mutton Continuous
x4_rice Continuous
x5_wheat Continuous
x6_gel Continuous

In [14]:
# The problem data is written to an .lp file
modelFull.writeLP("WhiskasModel.lp")

[x1_chicken, x2_beef, x3_mutton, x4_rice, x5_wheat, x6_gel]

In [15]:
# Solve the optimization problem
status = modelFull.solve()
status

print(f"status: {modelFull.status}, {LpStatus[modelFull.status]}")
print(f"objective: {modelFull.objective.value()}")
for var in modelFull.variables():
    print(f"{var.name}: {var.value()}")
for name, constraint in modelFull.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 0.52
x1_chicken: 0.0
x2_beef: 60.0
x3_mutton: 0.0
x4_rice: 0.0
x5_wheat: 0.0
x6_gel: 40.0
sum_to_100%: 0.0
protein_req: 4.0
fat_req: 0.0
fibre_req: -1.7
salt_req: -0.10000000000000003


Interpreting the results:
- The solver reached an optimal solution, with an optimal price of 0.52$ per can.
- The optimal solution (cheapest result, considering the nutrient requirements) consist of:   
 0% chicken, 60% beef, 0% mutton, 0% rice, 0% wheat and 40% gel.

 Slack variables:   
- the total weight requirement (=100g) is **exactly met**, with no surplus/slack
- the minimum protein requirement (8.0g) **is met, and exceeded** by a surplus of 4.0g.
- the minimumfat requirement (6.0g) **is exactly met**, with no surplus/slack
- the maximum fibre requirement (2.0g) **is broken**, it exceeded by a slack of 1.7g
- the maximum salt requirement (0.4g) **is broken**, it exceeded by a slack of 0.1g



## 3. Full Formulation
##### Again, after best practices 
- Use Lists and Loops: Instead of declaring variables one by one, use lists and loops to initialize and define them.
- Parameterize Constants: Put all the constants like costs, protein content, etc., in lists or dictionaries so that it's easier to change and maintain.
- Use Descriptive Comments: Include comments that explain why you are doing what you're doing.
- Avoid Magic Numbers: Replace hard-coded numbers with named constants whenever possible to make your code more readable and maintainable.

In [23]:
from pulp import LpMaximize, LpMinimize, LpProblem, LpStatus, lpSum, LpVariable

# Initialize the model
modelFull = LpProblem("WhiskasFull_problem", LpMinimize)

## Store variables in lists or dictionaries
# Ingredient names
ingredients = ['chicken', 'beef', 'mutton', 'rice', 'wheat', 'gel']

# Costs per ingredient (in the same order as ingredients)
costs = [0.013, 0.008, 0.010, 0.002, 0.005, 0.001]

# Nutritional information (protein, fat, fibre, salt)
protein = [0.100, 0.200, 0.150, 0.000, 0.040, 0.000]
fat = [0.080, 0.100, 0.110, 0.010, 0.010, 0.000]
fibre = [0.001, 0.005, 0.003, 0.100, 0.150, 0.000]
salt = [0.002, 0.005, 0.007, 0.002, 0.008, 0.000]

In [24]:
# Initialize decision variables
x = LpVariable.dicts("Ingr", ingredients, lowBound=0, cat='Continuous')
x

{'chicken': Ingr_chicken,
 'beef': Ingr_beef,
 'mutton': Ingr_mutton,
 'rice': Ingr_rice,
 'wheat': Ingr_wheat,
 'gel': Ingr_gel}

In [25]:
# just quick inspection for my part
print(x[ingredients[0]])
print(ingredients[0])

Ingr_chicken
chicken


In [26]:
## Version 1: Using indexing ##
# Objective function
modelFull += lpSum([costs[i] * x[ingredients[i]] for i in range(len(ingredients))]), "Total Cost"

# Constraints
modelFull += lpSum(x[ingredients[i]] for i in range(len(ingredients))) == 100, "PercentageSum"
modelFull += lpSum([protein[i] * x[ingredients[i]] for i in range(len(ingredients))]) >= 8.0, "ProteinRequirement"
modelFull += lpSum([fat[i] * x[ingredients[i]] for i in range(len(ingredients))]) >= 6.0, "FatRequirement"
modelFull += lpSum([fibre[i] * x[ingredients[i]] for i in range(len(ingredients))]) <= 2.0, "FibreRequirement"
modelFull += lpSum([salt[i] * x[ingredients[i]] for i in range(len(ingredients))]) <= 0.4, "SaltRequirement"

# ## Version 2: Zipping together lists ##
# # Objective function
# modelFull += lpSum(cost * x[ingredient] for cost, ingredient in zip(costs, ingredients)), "Total Cost"

# # Constraints
# modelFull += lpSum(x[ingredient] for ingredient in ingredients) == 100, "PercentageSum"
# modelFull += lpSum(prot * x[ingredient] for prot, ingredient in zip(protein, ingredients)) >= 8.0, "ProteinRequirement"
# modelFull += lpSum(fat * x[ingredient] for fat, ingredient in zip(fat, ingredients)) >= 6.0, "FatRequirement"
# modelFull += lpSum(fibre * x[ingredient] for fibre, ingredient in zip(fibre, ingredients)) <= 2.0, "FibreRequirement"
# modelFull += lpSum(salt * x[ingredient] for salt, ingredient in zip(salt, ingredients)) <= 0.4, "SaltRequirement"

modelFull

WhiskasFull_problem:
MINIMIZE
0.008*Ingr_beef + 0.013*Ingr_chicken + 0.001*Ingr_gel + 0.01*Ingr_mutton + 0.002*Ingr_rice + 0.005*Ingr_wheat + 0.0
SUBJECT TO
PercentageSum: Ingr_beef + Ingr_chicken + Ingr_gel + Ingr_mutton + Ingr_rice
 + Ingr_wheat = 100

ProteinRequirement: 0.2 Ingr_beef + 0.1 Ingr_chicken + 0.15 Ingr_mutton
 + 0.04 Ingr_wheat >= 8

FatRequirement: 0.1 Ingr_beef + 0.08 Ingr_chicken + 0.11 Ingr_mutton
 + 0.01 Ingr_rice + 0.01 Ingr_wheat >= 6

FibreRequirement: 0.005 Ingr_beef + 0.001 Ingr_chicken + 0.003 Ingr_mutton
 + 0.1 Ingr_rice + 0.15 Ingr_wheat <= 2

SaltRequirement: 0.005 Ingr_beef + 0.002 Ingr_chicken + 0.007 Ingr_mutton
 + 0.002 Ingr_rice + 0.008 Ingr_wheat <= 0.4

VARIABLES
Ingr_beef Continuous
Ingr_chicken Continuous
Ingr_gel Continuous
Ingr_mutton Continuous
Ingr_rice Continuous
Ingr_wheat Continuous

In [27]:
# Write to a file
modelFull.writeLP("WhiskasModel_best_practice.lp")

# Solve the problem
status = modelFull.solve()

print(f"status: {modelFull.status}, {LpStatus[modelFull.status]}")
print(f"objective: {modelFull.objective.value()}")
for var in modelFull.variables():
    print(f"{var.name}: {var.value()}")
for name, constraint in modelFull.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 0.52
Ingr_beef: 60.0
Ingr_chicken: 0.0
Ingr_gel: 40.0
Ingr_mutton: 0.0
Ingr_rice: 0.0
Ingr_wheat: 0.0
PercentageSum: 0.0
ProteinRequirement: 4.0
FatRequirement: 0.0
FibreRequirement: -1.7
SaltRequirement: -0.10000000000000003


Same answer as previously.